In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('../cache/merged.parquet')

In [3]:
df.head()

,Date,Actor1Country,Actor1GeoCountry,Actor1Type,Actor2Country,Actor2GeoCountry,Actor2Type,ActionCountry,EventType,QuadClass,...,Source,Timestamp,Actor1TypeGeneral,Actor2TypeGeneral,Magnitude,Impact,Impact_bin,pct_change_15min,pct_change_30min,pct_change_24h
0,2019-01-01,NaN,NaN,CVL,NaN,NaN,NaN,NaN,Diplom Coop,VerbalCoop,...,https://telegrafi.com/ne-shenj-proteste-labino...,1546300800,Other,None,6.85,2.33,Positive,-0.033061,-0.226363,-2.433464
1,2019-01-01,BRA,BR,NaN,NaN,NaN,NaN,BR,Intent to Coop,VerbalCoop,...,http://www.jornaldeluzilandia.com.br/txt.php?i...,1546300800,None,None,3.34,1.34,Slightly Positive,-0.033061,-0.226363,-2.433464
2,2019-01-01,NaN,US,GOV,NaN,US,GOV,US,Appeal,VerbalCoop,...,https://hanfordsentinel.com/news/national/govt...,1546300800,GOV,GOV,8.73,2.79,Positive,-0.033061,-0.226363,-2.433464
3,2019-01-01,USA,US,NaN,NaN,NaN,NaN,US,Disapprove,VerbalConf,...,http://midutahradio.com/news/national-news/lou...,1546300800,None,None,6.72,-1.34,Slightly Negative,-0.033061,-0.226363,-2.433464
4,2019-01-01,USA,US,EDU,NaN,NaN,NaN,US,Consult,VerbalCoop,...,http://www.q106dot5.com/news/blind-baker-launc...,1546300800,Benevolent,None,5.32,1.01,Slightly Positive,-0.033061,-0.226363,-2.433464


In [5]:
quants = [0, 0.5, 0.9, 0.99]
mag_quantiles = [df['Magnitude'].quantile(quant) for quant in quants]

df['AbsChange'] = df['pct_change_30min'].abs()
avg_change = df.AbsChange.mean()
catg_features = ['Actor1Country', 'Actor1GeoCountry', 'Actor1Type',
                 'Actor2Country', 'Actor2GeoCountry', 'Actor2Type',
                 'ActionCountry', 'EventType']

num_to_keep = 8

for feature in catg_features:
    for i, mag in enumerate(mag_quantiles):
        df_sub_orig = df[df.Magnitude > mag].astype(object)
        top_countries = df_sub_orig[feature].value_counts().index[:40]
        df_sub = df_sub_orig.groupby(feature).filter(lambda x: x.name in top_countries)
        df_grouped = (df_sub.groupby([feature], observed=True).AbsChange.mean() - avg_change).abs().sort_values(ascending=False)
        df_grouped = df_grouped[:num_to_keep]
        df_grouped = df_grouped.reset_index()
        print(df_grouped)
        

  Actor1Country AbsChange
0           VNM  0.095353
1           AZE  0.036359
2           ARM  0.035602
3           PER  0.034055
4           SYR  0.032665
5           PSE  0.032392
6           AUS  0.032247
7           IND  0.025443
  Actor1Country AbsChange
0           VNM  0.101154
1           AZE  0.046155
2           ARM  0.043357
3           PSE  0.043349
4           AUS  0.034767
5           UKR  0.031836
6           IND  0.030577
7           ISR   0.02906
  Actor1Country AbsChange
0           MMR  0.071231
1           AZE  0.070407
2           PSE    0.0679
3           UKR  0.060683
4           ISR  0.060015
5           KOR  0.052142
6           ARM   0.05145
7           IDN  0.047255
  Actor1Country AbsChange
0           PSE  0.122539
1           FIN  0.116801
2           IDN  0.116643
3           VNM  0.089135
4           IRQ  0.087503
5           SAU  0.083391
6           ISR     0.081
7           ESP  0.077223
  Actor1GeoCountry AbsChange
0               VM  0.094916
1     

KeyboardInterrupt: 